# Can we monitor what7 redditors say recently?

https://github.com/pushshift/api/blob/master/README.md

https://github.com/dmarx/psaw

In [ ]:
import requests
import json
import pandas as pd
from pmaw import PushshiftAPI

kw = 'GME'
subreddit='wallstreetbets'
obs_hrs = 24

curr_obs_end = pd.datetime.now().replace(minute=0, second=0, microsecond=0)
curr_obs_start = pd.datetime.timestamp(obs_end - pd.Timedelta(f"{obs_hrs}h"))

prev_obs_end = curr_obs_end - pd.Timedelta("24h")
prev_obs_start = curr_obs_start - 86400

api = PushshiftAPI()
curr_gen = api.search_comments(q=kw, subreddit=subreddit, 
                               after=int(curr_obs_start), 
                               before=int(curr_obs_end.timestamp()))
curr_df = pd.DataFrame([result for result in curr_gen])

prev_gen = api.search_comments(q=kw, subreddit=subreddit, 
                               after=int(prev_obs_start), 
                               before=int(prev_obs_end.timestamp()))
prev_df = pd.DataFrame([result for result in prev_gen])

In [ ]:
curr_df['set'] = "Today"
prev_df['set'] = "Yesterday"
prev_df['created_utc'] = prev_df['created_utc'] + 86400

df = pd.concat([curr_df, prev_df], ignore_index=True)

In [ ]:
df['created_ts'] = pd.to_datetime(df['created_utc'], unit='s', utc=True)
df['created_ts'] = df['created_ts'].dt.tz_convert('Asia/Hong_Kong')

In [ ]:
import plotly.express as px

hist = px.histogram(df, x='created_ts', color='set', barmode="overlay", #nbins=obs_hrs,
                    title=f"Occurrence of {kw} in r/{subreddit} in the last {obs_hrs} hours")
hist.show()